In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
os.chdir("/Users/quinnmackay/Documents/GitHub/BICC/Holcene Revision/")

In [23]:
### Loading all data

layer_file = pd.read_excel('New_layers.xlsx', usecols=[7,8])

new_layers = []
remove_layers = []
for i in range(len(layer_file["New_layer(m)"])): #cycle through new layers and put them in a list
    if pd.isna(layer_file["New_layer(m)"][i]): #drop NAs
        continue
    elif type(layer_file["New_layer(m)"][i]) == str: #split if string
        tempvals = layer_file["New_layer(m)"][i].split("-")
        for i, val in enumerate(tempvals): 
            new_layers.append(float(val))
    else: #append if float
        new_layers.append(layer_file["New_layer(m)"][i])

for i in range(len(layer_file["Remove_layer(yr)"])): #cycle through remove layers and put them in a list
    if pd.isna(layer_file["Remove_layer(yr)"][i]): #drop NAs
        continue
    elif type(layer_file["Remove_layer(yr)"][i]) == str: #split if string
        tempvals = layer_file["Remove_layer(yr)"][i].split("-")
        for i, val in enumerate(tempvals): 
            remove_layers.append(float(val))
    else: #append if float
        remove_layers.append(layer_file["Remove_layer(yr)"][i])

#load WD
WD_layer_count = pd.read_csv('WD2014 Layer Count.tab', comment="#", delimiter="\t", 
    names=["Depth ice/snow [m]",	"Cal age [ka BP] (ice age)", "Cal age std e [±] "
    "(ice age uncertainty due to an...)",	"Cal age std e [±] (ice age uncertainty due to CH...)",	
    "Gas age [ka BP] (gas age)",	"Age e [±] (gas age uncertainty (2 sigma))",	
    "Age diff [ka] (gas age-ice age difference (d...)",	"Age diff e [±] (delta age uncertainty (2 sigma))"])

WD_layer_count["Cal age [ka BP] (ice age)"] = WD_layer_count["Cal age [ka BP] (ice age)"] * 1000

WD_layer_count = WD_layer_count[WD_layer_count["Cal age [ka BP] (ice age)"]<=12000] #convert to holocene
print(len(WD_layer_count))

WDData = {
    'Depth(m)': WD_layer_count["Depth ice/snow [m]"],
    'Age(yr)': WD_layer_count["Cal age [ka BP] (ice age)"]
}
New_WDcount = pd.DataFrame(WDData)


12057


In [24]:
#removing layers and adjusting count

New_WDcount = New_WDcount[~New_WDcount["Age(yr)"].isin(remove_layers)] #removes layers from layer count that aren't included

progress_count = 0 #init prog counter
for i in range(len(New_WDcount["Age(yr)"]) - 1): #can't run last one since there can't be a gap after the last
    current = New_WDcount["Age(yr)"].iloc[i]
    next = New_WDcount["Age(yr)"].iloc[i+1]

    diff = next - current

    if abs(diff - 1) > 0.01:
        u = 0
        u += i+1 #using u to set the index to subtract 1 from (AKA: next)
        for s in range(len(New_WDcount["Age(yr)"][i+1:])): #this should loop for every number above the current (same as the len() in parent loop)
            New_WDcount.iloc[u, New_WDcount.columns.get_loc("Age(yr)")] -= 1 #subtracts 1 for every value above the current one that has the gap
            u+=1
    progress_count+=1 #for percentage bar

    perc_complete = (progress_count)/(len(New_WDcount["Age(yr)"])-1)

    if progress_count % (np.round(((len(New_WDcount["Age(yr)"]) - 1)/10),0)) == 0: #sets to show every 10 percent
        print(f"{np.round(perc_complete*100,0)}%")

#I think this all works now...should run code by someone to make sure

10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
100.0%


In [25]:
#adding new layers 

i=0
init_newWD_length = len(New_WDcount["Age(yr)"])
while len(New_WDcount["Age(yr)"]) < init_newWD_length+len(new_layers): #runs until all new layers are added
    
    current = New_WDcount["Depth(m)"].iloc[i] #get current and next depths for checking
    next = New_WDcount["Depth(m)"].iloc[i+1]

    add_depth = None
    for depth in new_layers: #check if layer needs to be added
        if depth > current and depth < next:
            add_depth = depth #sets the depth for which a new layer needs to be added

            new_age = New_WDcount["Age(yr)"].iloc[i] + 1

            new_row = {"Depth(m)": add_depth, "Age(yr)": new_age}
            new_row_df = pd.DataFrame([new_row])  # <-- create a 1-row DataFrame
            New_WDcount = pd.concat([New_WDcount, new_row_df], ignore_index=True)
            break

        else:
            continue

    
    New_WDcount = New_WDcount.sort_values(by="Depth(m)").reset_index(drop=True)
    i+=1



In [26]:
#adjusting duplicate ages

progress_count = 0 #init prog counter
for i in range(len(New_WDcount["Age(yr)"]) - 1): #can't run last one since there can't be a gap after the last
    current = New_WDcount["Age(yr)"].iloc[i]
    next = New_WDcount["Age(yr)"].iloc[i+1]

    diff = next - current

    if abs(diff) < 0.99: #if the difference is less than 1 year, then it is a duplicate
        u = 0
        u += i+1 #using u to set the index start the change from (AKA: next)
        for s in range(len(New_WDcount["Age(yr)"][i+1:])): #this should loop for every number above the current (same as the len() in parent loop)
            New_WDcount.iloc[u, New_WDcount.columns.get_loc("Age(yr)")] += 1 #subtracts 1 for every value above the current one that has the gap
            u+=1
    progress_count+=1 #for percentage bar

    perc_complete = (progress_count)/(len(New_WDcount["Age(yr)"])-1)

    if progress_count % (np.round(((len(New_WDcount["Age(yr)"]) - 1)/10),0)) == 0: #sets to show every 10 percent
        print(f"{np.round(perc_complete*100,0)}%")

10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%


In [27]:
#making names same as original
renamed_new_WDcount = New_WDcount.rename(columns={"Depth(m)": "#Depth ice/snow [m]", "Age(yr)": "Cal age [ka BP] (ice age)"})

#convert back to ka so its the same
renamed_new_WDcount["Cal age [ka BP] (ice age)"] = renamed_new_WDcount["Cal age [ka BP] (ice age)"] / 1000

#round to 3 decimal places #in case of small python math error
renamed_new_WDcount["Cal age [ka BP] (ice age)"] = renamed_new_WDcount["Cal age [ka BP] (ice age)"].round(3) 

renamed_new_WDcount.to_csv('Updated_WD2014 Layer Count.tab', sep="\t", index=False, header=True) #save to tab delimited file


In [28]:
#doing the error count (copied from GICC21-WDC_Modified Errors.py)

import pandas as pd
import numpy as np
import os
import re

#set data paths
os.chdir("/Users/quinnmackay/Documents/GitHub/BICC/Holcene Revision")
links_path = "GRIP_WDC_TephraLinks.xlsx"
WD2014_path = "Updated_WD2014 Layer Count.tab"
GICC_GRIP_age = "GICC05-GICC21 Conversion.xlsx"

#import data
links = pd.read_excel(links_path, usecols=[0,1,2,3,4,5,6,7,8], skiprows=29, names=["WDC(m)", "GRIP(m)", "WD2014 age (BC/AD iso)", "WD2014 age (yr BP)", "GICC05 age (yr BP)", "age diff", "GRIP dz/dt", "distance (yrs)", "type"])
WD2014 = pd.read_csv(WD2014_path, comment="#", delimiter="\t", names=["Depth ice/snow [m]", "Cal age [ka BP] (ice age)"])
GICC_GRIP_age = pd.read_excel(GICC_GRIP_age, sheet_name=1, skiprows=1, names=["Age (b2k)", "dt years", "age (CE/BCE)", "EastGRIP (m)",	"NEEM (m)",	"NorthGRIP1 (m)",	"NorthGRIP2 (m)",	"NEEM-2011-S1 (m)",	"GRIP (m)",	"DYE-3 79 (m)",	"DYE-3 4B (m)",	"DYE-3 18C (m)"])

GICC_GRIP_age["Age (b1950)"] = GICC_GRIP_age["Age (b2k)"] - 50 #make b1950 column

WDC_GICC_compare = links[["WDC(m)", "GRIP(m)"]].copy(deep=True) #create a new dataframe for WDC and GRIP depths using stratigraphic links

#get interp age for WDC depths from layer count
interp_wdcage = np.interp(WDC_GICC_compare["WDC(m)"], WD2014["Depth ice/snow [m]"], WD2014["Cal age [ka BP] (ice age)"]) #interpolate WDC depths to get ages
interp_wdcage = interp_wdcage * 1000 #convert ka BP to yr BP

WDC_GICC_compare["WD2014 age (yr BP1950)"] = interp_wdcage #add interpolated ages to dataframe

#get interp age for GRIP depths from GICC layer count
interp_giccage = np.interp(WDC_GICC_compare["GRIP(m)"], GICC_GRIP_age["GRIP (m)"], GICC_GRIP_age["Age (b1950)"]) #interpolate GRIP depths to get GICC ages
WDC_GICC_compare["GICC age (yr BP1950)"] = interp_giccage

# Filter out rows where GICC age > 3800
WDC_GICC_compare = WDC_GICC_compare[0:31]

WDC_GICC_compare["difference (yr)"] = WDC_GICC_compare["WD2014 age (yr BP1950)"] - WDC_GICC_compare["GICC age (yr BP1950)"] #calculate difference between ages


# this computes the error from that given section, which is the difference between layer counts in GICC and WDC for that section

section_error = np.zeros(len(WDC_GICC_compare)) #create empty list for section error
for i in range(0, len(WDC_GICC_compare)):
    if i == 0:
        WDC_error = WDC_GICC_compare["WD2014 age (yr BP1950)"][i] - 0 #calculate section error for each row
        GICC_error = WDC_GICC_compare["GICC age (yr BP1950)"][i] - 0 #calculate section error for each row
    else:
        WDC_error = WDC_GICC_compare["WD2014 age (yr BP1950)"][i] - WDC_GICC_compare["WD2014 age (yr BP1950)"][i-1] #calculate section error for each row
        GICC_error = WDC_GICC_compare["GICC age (yr BP1950)"][i] - WDC_GICC_compare["GICC age (yr BP1950)"][i-1] #calculate section error for each row
    section_error[i] = WDC_error - GICC_error #calculate section error for each row

WDC_GICC_compare["section error (yr)"] = section_error #calculate difference between ages

output_path = "Modified_WDC_GICC_Compare.xlsx"
WDC_GICC_compare.to_excel(output_path, index=False)

print(f"File saved to: {output_path}")

File saved to: Modified_WDC_GICC_Compare.xlsx
